In [1]:
%load_ext autoreload
%autoreload 2

from mimic_iv_analysis.io import DataLoader, TableNamesHOSP, TableNamesICU, ParquetConverter
import pandas as pd
import dask.dataframe as dd

In [2]:
from dask.distributed import Client
client = Client(n_workers=2, threads_per_worker=4, processes=True, memory_limit='8GB')
client

2025-05-15 21:40:08,372 - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2025-05-15 21:40:08,382 - INFO - State start
2025-05-15 21:40:08,383 - INFO - Found stale lock file and directory '/var/folders/yr/5jld3pmj3p75vsk6f7dsr6gm0000gn/T/dask-scratch-space/worker-vqp68ihz', purging
2025-05-15 21:40:08,384 - INFO - Found stale lock file and directory '/var/folders/yr/5jld3pmj3p75vsk6f7dsr6gm0000gn/T/dask-scratch-space/scheduler-v6zcioyd', purging
2025-05-15 21:40:08,384 - INFO - Found stale lock file and directory '/var/folders/yr/5jld3pmj3p75vsk6f7dsr6gm0000gn/T/dask-scratch-space/worker-4duvqm34', purging
2025-05-15 21:40:08,387 - INFO -   Scheduler at:     tcp://127.0.0.1:61215
2025-05-15 21:40:08,387 - INFO -   dashboard at:  http://127.0.0.1:8787/status
2025-05-15 21:40:08,387 - INFO - Registering Worker plugin shuffle
2025-05-15 21:40:08,406 - INFO -         Start Nanny at: 'tcp://127.0.0.1:61219'
20

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 8,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61215,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:61225,Total threads: 4
Dashboard: http://127.0.0.1:61228/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:61219,


2025-05-15 21:40:20,731 - WARNING - Shuffle 55e58c6a79be0ecd4d5e86477bbdcf7a initialized by task ('shuffle-transfer-55e58c6a79be0ecd4d5e86477bbdcf7a', 0) executed on worker tcp://127.0.0.1:61226
2025-05-15 21:40:21,240 - WARNING - Shuffle 55e58c6a79be0ecd4d5e86477bbdcf7a deactivated due to stimulus 'task-finished-1747359621.240308'
2025-05-15 22:17:01,552 - WARNING - Shuffle 726e316b7a881bc387b423f963373c38 initialized by task ('shuffle-transfer-726e316b7a881bc387b423f963373c38', 0) executed on worker tcp://127.0.0.1:61225
2025-05-15 22:17:01,568 - WARNING - Shuffle 726e316b7a881bc387b423f963373c38 deactivated due to stimulus 'task-finished-1747361821.567484'
2025-05-15 22:19:33,833 - WARNING - Shuffle 2c9612cf2c9a075a970f02975bf051f0 initialized by task ('shuffle-transfer-2c9612cf2c9a075a970f02975bf051f0', 0) executed on worker tcp://127.0.0.1:61226
2025-05-15 22:19:33,850 - WARNING - Shuffle 2c9612cf2c9a075a970f02975bf051f0 deactivated due to stimulus 'task-finished-1747361973.849893

## Scan the directory

In [3]:

data_loader = DataLoader()

# Scan the directory
data_loader.scan_mimic_directory()


data_loader.study_tables_info

2025-05-15 21:40:10,714 - INFO - Initializing Filtering class...


,module,table_name,file_path,file_size,display_name,suffix,columns_list
0,hosp,admissions,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,25736908,admissions 25.7 MB,.parquet,"{admit_provider_id, edouttime, edregtime, insu..."
5,hosp,d_icd_diagnoses,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,2683277,d_icd_diagnoses 2.7 MB,.parquet,"{icd_code, long_title, icd_version}"
10,hosp,poe,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,1793400412,poe 1.8 GB,.parquet,"{poe_id, poe_seq, order_provider_id, discontin..."
13,hosp,diagnoses_icd,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,75013787,diagnoses_icd 75.0 MB,.parquet,"{hadm_id, seq_num, icd_code, subject_id, icd_v..."
17,hosp,poe_detail,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,105773003,poe_detail 105.8 MB,.parquet,"{field_name, poe_id, poe_seq, subject_id, fiel..."
18,hosp,patients,/Users/artinmajdi/Documents/GitHubs/RAP/mimic_...,4582613,patients 4.6 MB,.parquet,"{anchor_year, dod, gender, subject_id, anchor_..."


In [35]:
%%time

subject_ids = data_loader.get_partial_subject_id_list_for_partial_loading(num_subjects=5)
print(subject_ids)

df = data_loader.load_table(table_name=TableNamesHOSP.ADMISSIONS, partial_loading=True, subject_ids=subject_ids)
df = df.compute()
df

2025-05-15 21:52:04,738 - INFO - Filtering admissions by subject_id for 5 subjects.


[10000032, 10000068, 10000084, 10000108, 10000117]
CPU times: user 27.7 ms, sys: 16 ms, total: 43.7 ms
Wall time: 181 ms


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,P49AFC,TRANSFER FROM HOSPITAL,HOME,Medicaid,English,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0.0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,P784FA,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0.0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,EW EMER.,P19UTS,EMERGENCY ROOM,HOSPICE,Medicaid,English,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0.0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,P06OTX,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0.0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaT,EU OBSERVATION,P39NWO,EMERGENCY ROOM,<NA>,<NA>,English,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0.0
5,10000084,23052089,2160-11-21 01:56:00,2160-11-25 14:52:00,NaT,EW EMER.,P42H7G,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,English,MARRIED,WHITE,2160-11-20 20:36:00,2160-11-21 03:20:00,0.0
6,10000084,29888819,2160-12-28 05:11:00,2160-12-28 16:07:00,NaT,EU OBSERVATION,P35NE4,PHYSICIAN REFERRAL,<NA>,Medicare,English,MARRIED,WHITE,2160-12-27 18:32:00,2160-12-28 16:07:00,0.0
7,10000108,27250926,2163-09-27 23:17:00,2163-09-28 09:04:00,NaT,EU OBSERVATION,P40JML,EMERGENCY ROOM,<NA>,<NA>,English,SINGLE,WHITE,2163-09-27 16:18:00,2163-09-28 09:04:00,0.0
8,10000117,22927623,2181-11-15 02:05:00,2181-11-15 14:52:00,NaT,EU OBSERVATION,P47EY8,EMERGENCY ROOM,<NA>,Medicaid,English,DIVORCED,WHITE,2181-11-14 21:51:00,2181-11-15 09:57:00,0.0
9,10000117,27988844,2183-09-18 18:10:00,2183-09-21 16:30:00,NaT,OBSERVATION ADMIT,P13ACE,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicaid,English,DIVORCED,WHITE,2183-09-18 08:41:00,2183-09-18 20:20:00,0.0


In [36]:
tables_dict = data_loader.load_all_study_tables(partial_loading=True, num_subjects=5)


2025-05-15 21:52:07,297 - INFO - Filtering admissions by subject_id for 5 subjects.
2025-05-15 21:52:07,301 - INFO - Table d_icd_diagnoses does not have a subject_id column. Partial loading is not possible. Skipping partial loading.
2025-05-15 21:52:07,305 - INFO - Filtering poe by subject_id for 5 subjects.
2025-05-15 21:52:07,309 - INFO - Filtering diagnoses_icd by subject_id for 5 subjects.
2025-05-15 21:52:07,313 - INFO - Filtering poe_detail by subject_id for 5 subjects.
2025-05-15 21:52:07,318 - INFO - Filtering patients by subject_id for 5 subjects.


In [86]:
print(tables_dict.keys())


patients_df        = tables_dict[TableNamesHOSP.PATIENTS.value]
admissions_df      = tables_dict[TableNamesHOSP.ADMISSIONS.value]
diagnoses_icd_df   = tables_dict[TableNamesHOSP.DIAGNOSES_ICD.value]
poe_df             = tables_dict[TableNamesHOSP.POE.value]
d_icd_diagnoses_df = tables_dict[TableNamesHOSP.D_ICD_DIAGNOSES.value]
poe_detail_df      = tables_dict[TableNamesHOSP.POE_DETAIL.value]

dict_keys(['admissions', 'd_icd_diagnoses', 'poe', 'diagnoses_icd', 'poe_detail', 'patients'])


In [135]:
df12 = patients_df.merge(admissions_df, on='subject_id', how='inner')
df34 = diagnoses_icd_df.merge(d_icd_diagnoses_df, on=('icd_code', 'icd_version'), how='inner')
df56 = poe_df.merge(poe_detail_df, on=('poe_id', 'poe_seq', 'subject_id'), how='left')

df1234   = df12.merge(df34, on=('subject_id', 'hadm_id'), how='inner')
df123456 = df1234.merge(df56, on=('subject_id', 'hadm_id'), how='inner')

In [136]:
df123456c = df123456.compute()
df123456c

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,hadm_id,admittime,dischtime,deathtime,...,ordertime,order_type,order_subtype,transaction_type,discontinue_of_poe_id,discontinued_by_poe_id,order_provider_id,order_status,field_name,field_value
0,10000032,F,52.0,2180.0,2014 - 2016,2180-09-09,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,...,2180-05-06 20:16:56,Lab,<NA>,New,<NA>,<NA>,P21AVM,Inactive,<NA>,<NA>
1,10000032,F,52.0,2180.0,2014 - 2016,2180-09-09,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,...,2180-05-06 20:37:14,Medications,<NA>,New,<NA>,10000032-14,P21AVM,Inactive,<NA>,<NA>
2,10000032,F,52.0,2180.0,2014 - 2016,2180-09-09,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,...,2180-05-06 20:37:31,Medications,<NA>,D/C,10000032-13,<NA>,P21AVM,Inactive,<NA>,<NA>
3,10000032,F,52.0,2180.0,2014 - 2016,2180-09-09,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,...,2180-05-06 20:50:49,Lab,<NA>,New,<NA>,<NA>,P21AVM,Inactive,<NA>,<NA>
4,10000032,F,52.0,2180.0,2014 - 2016,2180-09-09,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,...,2180-05-06 20:51:19,Lab,<NA>,New,<NA>,<NA>,P21AVM,Inactive,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4564,10000117,F,48.0,2174.0,2008 - 2010,<NA>,27988844,2183-09-18 18:10:00,2183-09-21 16:30:00,NaT,...,2183-09-19 11:13:02,ADT orders,Postop,New,<NA>,<NA>,P700FL,Inactive,Admit category,Admit to inpatient
4565,10000117,F,48.0,2174.0,2008 - 2010,<NA>,27988844,2183-09-18 18:10:00,2183-09-21 16:30:00,NaT,...,2183-09-19 11:13:02,ADT orders,Postop,New,<NA>,<NA>,P700FL,Inactive,Admit to,Surgery
4566,10000117,F,48.0,2174.0,2008 - 2010,<NA>,27988844,2183-09-18 18:10:00,2183-09-21 16:30:00,NaT,...,2183-09-19 11:13:02,Medications,<NA>,New,<NA>,<NA>,P700FL,Inactive,<NA>,<NA>
4567,10000117,F,48.0,2174.0,2008 - 2010,<NA>,27988844,2183-09-18 18:10:00,2183-09-21 16:30:00,NaT,...,2183-09-19 11:13:02,Medications,<NA>,New,<NA>,10000117-121,P700FL,Inactive,<NA>,<NA>


In [168]:
# Filter age
df123456 = df123456[(df123456.anchor_age >= 18.0) & (df123456.anchor_age <= 75.0)]

# Filter icd_version. Ony keep the ICD 10
df123456 = df123456[df123456.icd_version == 10]

# remove columns discontinued_by_poe_id
df123456 = df123456.drop(columns=['discontinued_by_poe_id'])


In [169]:
df = df123456.compute()
df.keys()

Index(['subject_id', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod', 'hadm_id', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admit_provider_id',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'marital_status', 'race', 'edregtime', 'edouttime',
       'hospital_expire_flag', 'seq_num', 'icd_code', 'icd_version',
       'long_title', 'poe_id', 'poe_seq', 'ordertime', 'order_type',
       'order_subtype', 'transaction_type', 'discontinue_of_poe_id',
       'order_provider_id', 'order_status', 'field_name', 'field_value'],
      dtype='object')

In [180]:
# how to check if either one or both of admittime or dischtime are null
admissions_df_full = data_loader.load_table(table_name=TableNamesHOSP.ADMISSIONS, partial_loading=False)

In [182]:
df = admissions_df_full.compute()
df

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,P49AFC,TRANSFER FROM HOSPITAL,HOME,Medicaid,English,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0.0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,P784FA,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0.0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,EW EMER.,P19UTS,EMERGENCY ROOM,HOSPICE,Medicaid,English,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0.0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,P06OTX,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0.0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaT,EU OBSERVATION,P39NWO,EMERGENCY ROOM,<NA>,<NA>,English,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546023,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,EW EMER.,P13JMH,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,English,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0.0
546024,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaT,EW EMER.,P38XL8,PHYSICIAN REFERRAL,HOME HEALTH CARE,Medicaid,English,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0.0
546025,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P33612,EMERGENCY ROOM,DIED,Private,English,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1.0
546026,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaT,EW EMER.,P036NA,EMERGENCY ROOM,HOME,Private,English,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0.0


In [183]:
df.deathtime.isnull().sum()

np.int64(534238)